<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220405_ji_hard%26soft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)


- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
valid = df[-100000:]
df = df[:900000]

In [7]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [8]:
df = df.fillna('')

In [9]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
899995,id_0899996,I,56,561,식당에서,접객시설을 갖추고,중식,식당에서 접객시설을 갖추고 중식,I 56 561
899996,id_0899997,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스,개인택시로 일반인을 대상으로 승객운송서비스,H 49 492
899997,id_0899998,P,85,855,학원에서,초중고생 대상,"국어,영어,수학","학원에서 초중고생 대상 국어,영어,수학",P 85 855
899998,id_0899999,P,85,851,초등학교,학생들을 대상으로,기초적인 초등교육,초등학교 학생들을 대상으로 기초적인 초등교육,P 85 851


In [10]:
train = df[['text', 'target']]
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...
899995,식당에서 접객시설을 갖추고 중식,I 56 561
899996,개인택시로 일반인을 대상으로 승객운송서비스,H 49 492
899997,"학원에서 초중고생 대상 국어,영어,수학",P 85 855
899998,초등학교 학생들을 대상으로 기초적인 초등교육,P 85 851


# 토크나이저, 함수

In [11]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [13]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,0
1,상점내에서 일반인을 대상으로 채소.과일판매,1
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,영업점에서 일반소비자에게 열쇠잠금장치,3
4,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...
899995,식당에서 접객시설을 갖추고 중식,6
899996,개인택시로 일반인을 대상으로 승객운송서비스,54
899997,"학원에서 초중고생 대상 국어,영어,수학",34
899998,초등학교 학생들을 대상으로 기초적인 초등교육,71


In [14]:
my_dict[''] = len(my_dict)

In [15]:
train[train['text'].isnull()]

,text,target


In [16]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [ ]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220404_inference/model' + str(fold), save_function=accelerator.save)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  my_file = Path("/content/drive/MyDrive/220404_inference/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/7820 [00:00<?, ?it/s]

  0%|          | 0/7820 [00:00<?, ?it/s]

  0%|          | 0/7820 [00:00<?, ?it/s]

...

  0%|          | 0/7820 [00:00<?, ?it/s]

  0%|          | 0/7820 [00:00<?, ?it/s]

  0%|          | 0/7820 [00:00<?, ?it/s]

  0%|          | 0/7820 [00:00<?, ?it/s]

epoch 1 train acc 0.3452685421994885
epoch 1 train acc 0.3459778612531969
epoch 1 train acc 0.3454783407928389
epoch 1 train acc 0.34649736253196933
epoch 1 train acc 0.34491887787723785
epoch 1 train acc 0.3444193574168798
epoch 1 train acc 0.3445991847826087
epoch 1 train acc 0.34573809143222506
epoch 1 test acc 0.6501116071428571
epoch 1 test acc 0.6460060586734694
epoch 1 test acc 0.6547353316326531
epoch 1 test acc 0.6455676020408163
epoch 1 test acc 0.6507892219387755
epoch 1 test acc 0.6464445153061225
epoch 1 test acc 0.6494339923469388
epoch 1 test acc 0.6490353954081632
epoch 2 train acc 0.7198089833759591
epoch 2 train acc 0.7153232896419437
epoch 2 train acc 0.7166120524296675
epoch 2 train acc 0.717571131713555
epoch 2 train acc 0.7159327046035806
epoch 2 train acc 0.7152533567774936
epoch 2 train acc 0.715153452685422
epoch 2 train acc 0.7146339514066496
epoch 2 test acc 0.7998246173469388
epoch 2 test acc 0.795280612244898
epoch 2 test acc 0.793327487244898
epoch 2 test 

# 추론

In [17]:
test = valid.fillna('')

In [18]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
900000,id_0900001,G,47,472,소매업,,"더덕,견과류등 건강식품판매","소매업 더덕,견과류등 건강식품판매",G 47 472
900001,id_0900002,P,85,851,초등학교,학생들을 대상으로,중학교 진학을위한 교육서비스,초등학교 학생들을 대상으로 중학교 진학을위한 교육서비스,P 85 851
900002,id_0900003,S,96,961,매장에서,일반인을 대상으로,손톱관리,매장에서 일반인을 대상으로 손톱관리,S 96 961
900003,id_0900004,G,47,476,영업장에서,고객대상,볼링용품판매,영업장에서 고객대상 볼링용품판매,G 47 476
900004,id_0900005,G,46,467,노점에서,일반 소비자에게,원단(양복지),노점에서 일반 소비자에게 원단(양복지),G 46 467
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [21]:
test = test[['text', 'target']]
test = test.reset_index(drop=True)

In [22]:
test

,text,target
0,"소매업 더덕,견과류등 건강식품판매",G 47 472
1,초등학교 학생들을 대상으로 중학교 진학을위한 교육서비스,P 85 851
2,매장에서 일반인을 대상으로 손톱관리,S 96 961
3,영업장에서 고객대상 볼링용품판매,G 47 476
4,노점에서 일반 소비자에게 원단(양복지),G 46 467
...,...,...
99995,제품입고 워싱 청바지워싱,C 13 134
99996,현장에서 고객의요청에의해 실내인테리어,F 42 424
99997,영업점에서 일반소비자에게 여성의류 판매,G 47 474
99998,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [23]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220331/model' + str(fold), num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220331/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220331/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

In [24]:
print(my_variable)

NameError: ignored

In [25]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [91]:
valid

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
900000,id_0900001,G,47,472,소매업,NaN,"더덕,견과류등 건강식품판매"
900001,id_0900002,P,85,851,초등학교,학생들을 대상으로,중학교 진학을위한 교육서비스
900002,id_0900003,S,96,961,매장에서,일반인을 대상으로,손톱관리
900003,id_0900004,G,47,476,영업장에서,고객대상,볼링용품판매
900004,id_0900005,G,46,467,노점에서,일반 소비자에게,원단(양복지)
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [26]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331/pred_label4.csv')

In [27]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [28]:
df.head(3)

,text,target,label0,label1,label2,label3,label4
0,"소매업 더덕,견과류등 건강식품판매",G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,초등학교 학생들을 대상으로 중학교 진학을위한 교육서비스,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,매장에서 일반인을 대상으로 손톱관리,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961


In [29]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

0.94845

In [30]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

,text,target,label0,label1,label2,label3,label4
53,도장 도장.열쇠 도장제조,C 33 339,C 25 259,C 25 259,C 25 259,C 33 339,C 25 259
76,플락스틱 사출 의료기부품,C 22 222,C 26 262,C 22 222,C 22 222,C 22 222,C 22 222
86,레이온.PE 롤가공 부직포,C 13 139,C 13 132,C 13 132,C 13 132,C 22 222,C 22 222
97,작업장에서 과일동결건조 농산물 가공 식품 제조 가공,C 10 103,C 10 103,C 10 106,C 10 107,C 10 103,C 10 103
130,"반제품, 모니터 반제품입고, 조립",C 27 271,C 28 281,C 26 262,C 26 262,C 26 262,C 26 262
...,...,...,...,...,...,...,...
99899,"합판 합판, 재단",C 32 320,C 16 162,C 16 162,C 16 162,C 16 162,C 32 320
99913,녹음시설운영업,J 59 592,J 58 581,J 58 581,J 58 581,J 59 591,J 58 581
99968,"사업장에서 소매업자에게 소금,설탕,사료부원료,도매",G 46 463,G 46 463,G 46 462,G 46 462,G 46 462,G 46 462
99993,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스",C 32 320,C 22 222,C 22 222,C 15 151,C 22 222,C 22 222


In [31]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331/pred_prob4.csv')

In [32]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,215,216,217,218,219,220,221,222,223,224
0,"소매업 더덕,견과류등 건강식품판매",G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,3.777237e-06,0.973840,1.433160e-05,...,1.010151e-06,5.005155e-08,1.471048e-07,3.840608e-08,3.266160e-08,1.131977e-07,1.353871e-07,1.286196e-07,3.172463e-08,2.344985e-08
1,초등학교 학생들을 대상으로 중학교 진학을위한 교육서비스,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,5.358291e-07,0.000116,4.054415e-05,...,1.619573e-06,3.234700e-06,5.726298e-07,3.577608e-07,3.020433e-07,4.708298e-07,1.093865e-06,1.368612e-07,1.136648e-07,6.048967e-08
2,매장에서 일반인을 대상으로 손톱관리,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,2.022496e-06,0.000019,1.053330e-07,...,8.072614e-08,1.108134e-08,1.192671e-08,4.129781e-09,1.501855e-09,1.881710e-08,7.668867e-09,1.974253e-08,4.239185e-09,2.775183e-09


In [33]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,216,217,218,219,220,221,222,223,224,soft
0,"소매업 더덕,견과류등 건강식품판매",G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,3.777237e-06,0.973840,1.433160e-05,...,5.005155e-08,1.471048e-07,3.840608e-08,3.266160e-08,1.131977e-07,1.353871e-07,1.286196e-07,3.172463e-08,2.344985e-08,G 47 472
1,초등학교 학생들을 대상으로 중학교 진학을위한 교육서비스,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,5.358291e-07,0.000116,4.054415e-05,...,3.234700e-06,5.726298e-07,3.577608e-07,3.020433e-07,4.708298e-07,1.093865e-06,1.368612e-07,1.136648e-07,6.048967e-08,P 85 851
2,매장에서 일반인을 대상으로 손톱관리,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,2.022496e-06,0.000019,1.053330e-07,...,1.108134e-08,1.192671e-08,4.129781e-09,1.501855e-09,1.881710e-08,7.668867e-09,1.974253e-08,4.239185e-09,2.775183e-09,S 96 961


In [34]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

,label0,label1,label2,label3,label4,soft
0,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961
3,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476
4,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
...,...,...,...,...,...,...
99995,C 13 134,C 13 134,C 13 134,C 15 151,C 14 141,C 13 134
99996,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424
99997,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
99998,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856


In [180]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [181]:
soft[soft['soft'] != soft['hard']]

,label0,label1,label2,label3,label4,soft,temp,hard
306,M 73 732,M 73 732,M 73 732,F 42 424,M 73 732,F 42 424,"{'M 73 732': 4, 'F 42 424': 1}",M 73 732
363,C 26 262,C 27 271,C 26 262,C 28 281,C 27 272,C 28 281,"{'C 26 262': 2, 'C 27 271': 1, 'C 28 281': 1, ...",0
397,S 95 953,S 95 953,S 95 952,S 95 952,S 95 953,S 95 952,"{'S 95 953': 3, 'S 95 952': 2}",S 95 953
484,S 94 949,S 94 949,S 94 949,S 94 941,S 94 941,S 94 941,"{'S 94 949': 3, 'S 94 941': 2}",S 94 949
511,P 85 855,P 85 855,R 90 902,R 90 902,P 85 857,P 85 855,"{'P 85 855': 2, 'R 90 902': 2, 'P 85 857': 1}",0
...,...,...,...,...,...,...,...,...
99538,L 68 682,L 68 682,F 41 411,F 41 411,F 42 422,F 41 411,"{'L 68 682': 2, 'F 41 411': 2, 'F 42 422': 1}",0
99544,G 46 467,G 46 467,G 46 467,C 20 202,C 22 222,C 22 222,"{'G 46 467': 3, 'C 20 202': 1, 'C 22 222': 1}",G 46 467
99634,H 50 501,N 76 761,H 49 494,N 75 751,N 75 752,N 75 751,"{'H 50 501': 1, 'N 76 761': 1, 'H 49 494': 1, ...",0
99642,G 47 471,G 47 472,G 47 472,G 47 471,G 47 472,G 47 471,"{'G 47 471': 2, 'G 47 472': 3}",G 47 472


In [182]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

,label0,label1,label2,label3,label4,soft,hard
0,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961
3,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476
4,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
...,...,...,...,...,...,...,...
99995,C 13 134,C 13 134,C 13 134,C 15 151,C 14 141,C 13 134,C 13 134
99996,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424
99997,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
99998,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856


In [183]:
temp = df[df['soft'] != df['hard']]
temp

,label0,label1,label2,label3,label4,soft,hard
306,M 73 732,M 73 732,M 73 732,F 42 424,M 73 732,F 42 424,M 73 732
363,C 26 262,C 27 271,C 26 262,C 28 281,C 27 272,C 28 281,0
397,S 95 953,S 95 953,S 95 952,S 95 952,S 95 953,S 95 952,S 95 953
484,S 94 949,S 94 949,S 94 949,S 94 941,S 94 941,S 94 941,S 94 949
511,P 85 855,P 85 855,R 90 902,R 90 902,P 85 857,P 85 855,0
...,...,...,...,...,...,...,...
99538,L 68 682,L 68 682,F 41 411,F 41 411,F 42 422,F 41 411,0
99544,G 46 467,G 46 467,G 46 467,C 20 202,C 22 222,C 22 222,G 46 467
99634,H 50 501,N 76 761,H 49 494,N 75 751,N 75 752,N 75 751,0
99642,G 47 471,G 47 472,G 47 472,G 47 471,G 47 472,G 47 471,G 47 472


In [184]:
# 0이면 소프트 다르면 하드

temp['best'] = temp['hard']

for i in range(len(temp)):
   if temp['hard'].iloc[i] == 0:
     temp['best'].iloc[i] = temp['soft'].iloc[i]

temp

,label0,label1,label2,label3,label4,soft,hard,best
306,M 73 732,M 73 732,M 73 732,F 42 424,M 73 732,F 42 424,M 73 732,M 73 732
363,C 26 262,C 27 271,C 26 262,C 28 281,C 27 272,C 28 281,0,C 28 281
397,S 95 953,S 95 953,S 95 952,S 95 952,S 95 953,S 95 952,S 95 953,S 95 953
484,S 94 949,S 94 949,S 94 949,S 94 941,S 94 941,S 94 941,S 94 949,S 94 949
511,P 85 855,P 85 855,R 90 902,R 90 902,P 85 857,P 85 855,0,P 85 855
...,...,...,...,...,...,...,...,...
99538,L 68 682,L 68 682,F 41 411,F 41 411,F 42 422,F 41 411,0,F 41 411
99544,G 46 467,G 46 467,G 46 467,C 20 202,C 22 222,C 22 222,G 46 467,G 46 467
99634,H 50 501,N 76 761,H 49 494,N 75 751,N 75 752,N 75 751,0,N 75 751
99642,G 47 471,G 47 472,G 47 472,G 47 471,G 47 472,G 47 471,G 47 472,G 47 472


In [185]:
df

,label0,label1,label2,label3,label4,soft,hard
0,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961
3,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476
4,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
...,...,...,...,...,...,...,...
99995,C 13 134,C 13 134,C 13 134,C 15 151,C 14 141,C 13 134,C 13 134
99996,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424
99997,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
99998,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856


In [186]:
temp.index

Int64Index([  306,   363,   397,   484,   511,   734,   777,   817,   931,
             1178,
            ...
            98870, 98968, 99037, 99299, 99348, 99538, 99544, 99634, 99642,
            99817],
           dtype='int64', length=862)

In [187]:
temp

,label0,label1,label2,label3,label4,soft,hard,best
306,M 73 732,M 73 732,M 73 732,F 42 424,M 73 732,F 42 424,M 73 732,M 73 732
363,C 26 262,C 27 271,C 26 262,C 28 281,C 27 272,C 28 281,0,C 28 281
397,S 95 953,S 95 953,S 95 952,S 95 952,S 95 953,S 95 952,S 95 953,S 95 953
484,S 94 949,S 94 949,S 94 949,S 94 941,S 94 941,S 94 941,S 94 949,S 94 949
511,P 85 855,P 85 855,R 90 902,R 90 902,P 85 857,P 85 855,0,P 85 855
...,...,...,...,...,...,...,...,...
99538,L 68 682,L 68 682,F 41 411,F 41 411,F 42 422,F 41 411,0,F 41 411
99544,G 46 467,G 46 467,G 46 467,C 20 202,C 22 222,C 22 222,G 46 467,G 46 467
99634,H 50 501,N 76 761,H 49 494,N 75 751,N 75 752,N 75 751,0,N 75 751
99642,G 47 471,G 47 472,G 47 472,G 47 471,G 47 472,G 47 471,G 47 472,G 47 472


In [188]:
len(temp) - len(temp[temp['hard']==0])

490

In [189]:
df['best'] = df['soft']

In [190]:
df.iloc[306]

label0    M 73 732
label1    M 73 732
label2    M 73 732
label3    F 42 424
label4    M 73 732
soft      F 42 424
hard      M 73 732
best      F 42 424
Name: 306, dtype: object

In [191]:
for idx in temp.index:
  df['best'].loc[idx] = temp['best'].loc[idx]

In [192]:
df.iloc[306]

label0    M 73 732
label1    M 73 732
label2    M 73 732
label3    F 42 424
label4    M 73 732
soft      F 42 424
hard      M 73 732
best      M 73 732
Name: 306, dtype: object

In [193]:
df

,label0,label1,label2,label3,label4,soft,hard,best
0,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961
3,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476
4,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
...,...,...,...,...,...,...,...,...
99995,C 13 134,C 13 134,C 13 134,C 15 151,C 14 141,C 13 134,C 13 134,C 13 134
99996,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424
99997,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
99998,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856


In [194]:
valid['target'] = valid['digit_1'].astype(str) + " " + valid['digit_2'].astype(str) + " " + valid['digit_3'].astype(str)

valid

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target
0,id_0900001,G,47,472,소매업,NaN,"더덕,견과류등 건강식품판매",G 47 472
1,id_0900002,P,85,851,초등학교,학생들을 대상으로,중학교 진학을위한 교육서비스,P 85 851
2,id_0900003,S,96,961,매장에서,일반인을 대상으로,손톱관리,S 96 961
3,id_0900004,G,47,476,영업장에서,고객대상,볼링용품판매,G 47 476
4,id_0900005,G,46,467,노점에서,일반 소비자에게,원단(양복지),G 46 467
...,...,...,...,...,...,...,...,...
99995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134
99996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424
99997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474
99998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856


In [195]:
valid = valid.reset_index(drop=True)

In [196]:
df['target'] = valid['target']
df

,label0,label1,label2,label3,label4,soft,hard,best,target
0,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472,G 47 472
1,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851,P 85 851
2,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961,S 96 961
3,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476,G 47 476
4,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 46 467
...,...,...,...,...,...,...,...,...,...
99995,C 13 134,C 13 134,C 13 134,C 15 151,C 14 141,C 13 134,C 13 134,C 13 134,C 13 134
99996,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424,F 42 424
99997,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474,G 47 474
99998,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856,P 85 856


In [197]:
print(len(df[df['best'] == valid['target']])/len(df))
print(len(df[df['soft'] == valid['target']])/len(df))
print(len(df[df['hard'] == valid['target']])/len(df))

0.90657
0.90683
0.90579


In [198]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [199]:
print(my_variable)

NameError: ignored

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220331_inference/submission.csv', index=False)